In [0]:
import xml.etree.cElementTree as ET
import random
import json
import tokenization 
import torch
import sys
import torch.nn.functional as F
from modeling import BertConfig, BertForSequenceMultiClassification
from optimization import BERTAdam
import pickle

In [0]:
with open("train.json", "r") as read_file:
    train = json.load(read_file)
with open("test.json", "r") as read_file:
    test = json.load(read_file)

In [35]:
categories = []
polarity = []
for anno in train['annotation']:
    for cat in anno['category']:
        categories.append(cat)
    for pol in anno['polarity']:
        polarity.append(pol)
num_classes = len(set(categories))
print(num_classes)
categories = list(set(categories))
print(categories)
num_pol = len(set(polarity))
print(num_pol)
polarity = list(set(polarity))
print(polarity)

12
['LOCATION#GENERAL', 'AMBIENCE#GENERAL', 'DRINKS#PRICES', 'RESTAURANT#PRICES', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#MISCELLANEOUS', 'DRINKS#QUALITY', 'RESTAURANT#GENERAL', 'FOOD#QUALITY', 'DRINKS#STYLE_OPTIONS', 'FOOD#PRICES', 'SERVICE#GENERAL']
5
['negative', '', 'conflict', 'neutral', 'positive']


In [36]:
train['annotation'][1]

{'category': ['AMBIENCE#GENERAL', 'AMBIENCE#GENERAL'],
 'polarity': ['positive', 'positive'],
 'text': 'Он был в уютном уголке в конце главного зала, приглушенный свет это основная часть этого ресторана там нет дневного освещения это было большим плюсом для нашего дня рожденья!'}

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, label):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.label = label

In [0]:
tokenizer = tokenization.FullTokenizer(vocab_file='vocab.txt', do_lower_case=False)

In [0]:
def convert_examples_to_features(data, max_seq_length, tokenizer, num_classes, categories):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for idx in range(len(data['annotation'])):
        tokens_a = tokenizer.tokenize(data['annotation'][idx]['text'])
        
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]
            
        tokens = []
        tokens.append("[CLS]")
        for token in tokens_a:
            tokens.append(token)
        tokens.append("[SEP]")

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        
        label = [0 for i in range(num_classes)]
        for i in range(len(data['annotation'][idx]['category'])):
            ind = categories.index(data['annotation'][idx]['category'][i])
            inde = polarity.index(data['annotation'][idx]['polarity'][i])
            label[ind] = inde + 1
            
        features.append(
                InputFeatures(
                        input_ids=input_ids,
                        input_mask=input_mask,
                        label=label))
    return features

In [0]:
class Dataload(torch.utils.data.Dataset):
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return torch.LongTensor(self.features[index].input_ids),\
               torch.LongTensor(self.features[index].input_mask),\
               torch.LongTensor(self.features[index].label)

In [0]:
features = convert_examples_to_features(train, 512, tokenizer, num_classes, categories)
dataset_train = Dataload(features)
train_dataloader = torch.utils.data.DataLoader(dataset_train,batch_size = 8, shuffle=True,\
                                               num_workers=6, pin_memory=True)
features = convert_examples_to_features(test, 512, tokenizer, num_classes, categories)
dataset_test = Dataload(features)
test_dataloader = torch.utils.data.DataLoader(dataset_test,batch_size = 1, shuffle=False,\
                                               num_workers=6, pin_memory=True)

In [43]:
next(iter(train_dataloader))[2][0]

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


tensor([0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0])

In [29]:
#!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
#!tar -xf rubert_cased_L-12_H-768_A-12_pt.tar.gz

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
device = 'cuda'
bert_config = BertConfig.from_json_file('bert_config.json')
model = BertForSequenceMultiClassification(bert_config, num_classes)
model.bert.load_state_dict(torch.load('/content/drive/My Drive/pytorch_model.bin', map_location='cpu'))
model.to(device)
model = torch.nn.DataParallel(model)

RuntimeError: ignored

In [0]:
num_epoch = 10
no_decay = ['bias', 'gamma', 'beta']
optimizer_parameters = [
     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
     ]
num_train_steps = len(test_dataloader) * num_epoch
optimizer = BERTAdam(optimizer_parameters,
                     lr=5e-5,
                     warmup=0.1,
                     t_total=num_train_steps)

In [0]:
global_step = 0
for _ in range(num_epoch):
    model.train()
    total_loss = 0
    total_step = 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, label = batch
        loss, logits = model(input_ids, None, input_mask, label)
        loss = loss.mean()
        total_loss += loss.item()
        loss.backward()
        log = '\r {ep:d}/{len_d:d} {loss:.4f}\r'.format(ep = step, len_d=len(train_dataloader), loss=loss.item())
        sys.stdout.write(log)
        sys.stdout.flush()
#         if step%10 == 0:
#             print(loss.item(), loss_pol.item())
        if (step + 1) % 1 == 0:
            optimizer.step()    # We have accumulated enought gradients
            model.zero_grad()
            global_step += 1
        total_step += 1
    print(total_loss/total_step)
    total_loss = 0    
    total_step = 0
    acc = [0 for _ in range(num_classes)]  
    model.eval()
    for step, batch in enumerate(test_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, label = batch
        
        with torch.no_grad():
            logits = model(input_ids, None, input_mask)
        for i, ex in enumerate(logits):
            ex = F.softmax(ex, dim=-1)
            idx = ex.argmax()
            if idx == label[0][i].item():
                acc[i] += 1
        total_step += 1
    mean = []
    for i in range(num_classes):
        mean.append(acc[i]/total_step)
        print(categories[i],acc[i]/total_step)
    print(sum(mean)/num_classes)
    print('================')

In [0]:
# save the model to disk
filename = 'new_final_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [0]:
def predict(text, filename):
  final_model = pickle.load(open(filename, 'rb'))
  final_model.eval()
  tokens_a = tokenizer.tokenize(text)
  if len(tokens_a) > 512 - 2:
      tokens_a = tokens_a[0:(512 - 2)]
              
  tokens = []
  tokens.append("[CLS]")
  for token in tokens_a:
      tokens.append(token)
  tokens.append("[SEP]")

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  input_mask = [1] * len(input_ids)
  while len(input_ids) < 512:
      input_ids.append(0)
      input_mask.append(0)
  input_ids = torch.LongTensor(input_ids).unsqueeze(0)
  input_mask = torch.LongTensor(input_mask).unsqueeze(0)
  logits = final_model(input_ids, None, input_mask)
  for i, ex in enumerate(logits):
      ex = F.softmax(ex, dim=-1)
      idx = ex.argmax()
      if idx>0:
          print(categories[i], polarity[idx-1])

In [0]:
filename = 'new_final_model.sav'
predict('Еда, конечно, хороша, но вот обслуживание оставляет желать лучшего', filename)

In [0]:
predict('Обслуживание было выще всяческих похвал', filename)

In [0]:
predict('Атмосфера в ресторане была удручающей, пока нам не принесли оригинальные коктейли, вкус которых позволил нам насладиться богатым разнообразием кухни сполна', filename)

In [0]:
predict('В следующий раз, когда захочу испортить себе вечер, обязательно приду в этот ресторан', filename)

In [0]:
predict('Если хотите испортить себе настроение, приходите в этот ресторан', filename)

In [0]:
predict('Соли в суп могли бы и поменьше добавить', filename)

In [0]:
predict('По вкусу такое ощущение как-будто повар в первый раз готовил это блюдо', filename)

In [0]:
predict('Чек вышел средний по Москве', filename)

In [0]:
predict('Отдали дофига бабла', filename)

In [0]:
predict('Несмотря на изысканное оформление блюд, цены нас очень даже порадовали', filename)

In [0]:
predict('Вид на Москву-реку испортил весь аппетит', filename)

In [0]:
predict('Все было отлично, пока официанта вырвало прямо на меня', filename)

In [0]:
predict('Отдал 5000 р за два блюда, кошмар', filename)

In [0]:
predict('Еда дешевая, напитки дорогие', filename)

In [0]:
predict('Все классно, но музыку могли бы и потише сделать', filename)

In [0]:
predict('Больше не приду', filename)

In [0]:
predict('У официантки не было глаза', filename)

In [0]:
predict('Мужчина нашел волос в хлебе', filename)

In [0]:
predict('Из-за короновируса заведение было закрыто', filename)

In [0]:
predict('Отказали на мою просьбу позвать администратора', filename)

In [0]:
predict('Ром кола стоит копейки', filename)

In [0]:
predict('Барашек просто пэрсик', filename)

In [0]:
predict('Официантка Зоя просто вах', filename)

In [0]:
predict('Стейк был недожарен', filename)

In [0]:
predict('Стейк был дожарен', filename)